In [4]:
import datetime as dt
import utils
import pandas as pd
import icharts
from functools import cache


TEST_START = dt.datetime.strptime("2023-01-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2023-12-31", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = utils.INTERVAL_MIN1
EXCHANGE = utils.EXCHANGE_NSE
pickle_file_name = "ocdf_2024_02_17.pkl"

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 200)
ocdf = pd.read_pickle(pickle_file_name)

# Time for truth
ocdf = pd.read_pickle("test_analyzer_ocdf_2024_02_17.pkl")

In [5]:
ocdf["ce_ac_ex_diff"] = ocdf["ce_actual_chg_pc"] - ocdf["ec_ce_pc"]
ocdf["pe_ac_ex_diff"] = ocdf["pe_actual_chg_pc"] - ocdf["ec_pe_pc"]
ocdf["ce_first_candle_high_pc"] = (ocdf["ce_first_candle_high"] - ocdf["ce_first_candle_open"]) / ocdf["ce_first_candle_open"]
ocdf["ex_ce_first_candle_high_ch_pt"] = ocdf["first_candle_change_pt"] * ocdf["ce_delta"]
ocdf["first_candle_ac_ex_diff_pt"] = ocdf["ce_first_candle_chg_pt"] - ocdf["ex_ce_first_candle_high_ch_pt"] # first candle actual - expected diff of change
# ocdf["ce_oi"] = ocdf["ce_oi"].apply(human_readable_format)
# ocdf["pe_oi"] = ocdf["pe_oi"].apply(human_readable_format)

# Goal
Goal of this exercise is to
1. Backtest and make sure that strategy works
2. Identify strike where people most panic when market changes
3. Identify entry and exit points for the trade

In [69]:
# print(f"before filtering size: {ocdf.shape[0]}")
# put_ocdf_nona = ocdf.loc[ocdf.ec_pe_pc.notna()]
# print(f"call notna size: {call_ocdf_nona.shape[0]}")
# first_green_candle = ocdf[(ocdf.ec_ce_pc > .1) | (ocdf.ec_pe_pc > .1)]
# print(f"green size: {first_green_candle.shape[0]}")
# changed_more_than_expected = ocdf[(ocdf.ec_ce_pc < ocdf.ce_actual_chg_pc) | (ocdf.ec_pe_pc < ocdf.pe_actual_chg_pc)]
# print(f"more than expected size: {first_green_candle.shape[0]}")

def human_readable_format(x):
    # Define suffixes for different orders of magnitude
    suffixes = ['K', 'M', 'B', 'T']
    # suffixes = ['K', 'M', 'B', 'T']
    # Iterate over suffixes and divide x by 1000 until it's less than 1000
    for suffix in suffixes:
        if abs(x) < 1000:
            break
        x /= 1000
    # Format x with the appropriate suffix
    return f'{x:.2f}{suffix}'

## Let's Analyze calls here, forget puts
Dividing study of calls in two sections:
1. When market goes up and call sellers start to square off their positions driving the market up, creates an opportunity for me to buy
2. When market goes down and call buyers start to square off driving the prices down, creates opportunity for me to sell

### Market Goes Up

In [8]:
print(f"before filtering size: {ocdf.shape[0]}")
MINIMUM_MARKET_UP = 40
market_up = ocdf.loc[ocdf.market_open_pt > MINIMUM_MARKET_UP]
print(f"market up size: {market_up.shape[0]}")

# positive_first_candle = market_up.loc[market_up.first_candle_change_pt > 0]
# print(f"positive first candle size: {positive_first_candle.shape[0]}")

# negative_first_candle = market_up.loc[market_up.first_candle_change_pt < 0]
# print(f"negative first candle size: {negative_first_candle.shape[0]}")

MIN_PREMIUM = 90
min_premium_df = market_up.loc[market_up.ce_ltp >= MIN_PREMIUM]
print(f"min_premium_df size: {min_premium_df.shape[0]}")

# Actual - expected diff is positive
ac_ex_diff_pos = min_premium_df.loc[min_premium_df.first_candle_ac_ex_diff_pt > 0]
print(f"diff positive size: {ac_ex_diff_pos.shape[0]}")

MIN_VOLUME_REQUIRED = 10000
# Minimum liquidity required
min_volume = ac_ex_diff_pos.loc[ac_ex_diff_pos.ce_first_candle_volume > MIN_VOLUME_REQUIRED]
print(f"min_volume size: {min_volume.shape[0]}")

# # To study the nature, market's first candle is negative and premium still is green
# market_red = ac_ex_diff_pos.loc[ac_ex_diff_pos.first_candle_change_pt < 0]
# print(f"market_red size: {market_red.shape[0]}")

# more_than_ex = ocdf.loc[ocdf.ec_ce_pc < ocdf.ce_actual_chg_pc]
# print(f"more than ex size: {more_than_ex.shape[0]}")
# # positive_actual = ocdf.loc[(ocdf.ec_ce_pc < 0) & (ocdf.ce_actual_chg_pc > 0)]
# positive_actual = more_than_ex.loc[(more_than_ex.ce_actual_chg_pc > 0)]
# print(f"positive actual: {positive_actual.shape[0]}")
# # First premium candle is green
# first_green = positive_actual.loc[positive_actual.ce_first_candle_chg_pc > 0]
# print(f"first green size: {first_green.shape[0]}")

# Group by Strike Price and Trade date and order by ce_first_candle_chg_pc
# call_max_ch = pd.DataFrame(columns=["trade_date", "expiry", "strike_price", "ce_first_candle_chg_pc"])
call_max_ch = pd.DataFrame(columns=min_volume.columns)
call_grouped = min_volume.groupby(["trade_date", "expiry"])
for name, call_group in call_grouped:
    max_sorted = call_group.sort_values(by="ce_ac_ex_diff", ascending=True)
    # max_sorted = call_group.sort_values(by="ce_first_candle_chg_pc", ascending=False)
    max_row1 = max_sorted.iloc[0]
    call_max_ch = pd.concat([call_max_ch, max_row1.to_frame().T])
    if max_sorted.shape[0] > 1:
        max_row2 = max_sorted.iloc[1]
        call_max_ch = pd.concat([call_max_ch, max_row2.to_frame().T])
    if max_sorted.shape[0] > 2:
        max_row3 = max_sorted.iloc[2]
        call_max_ch = pd.concat([call_max_ch, max_row3.to_frame().T])

# call_max_ch = first_green.groupby(["trade_date", "expiry"])['ce_first_candle_chg_pc'].max()
print(f"size: {call_max_ch.shape[0]}")
# call_max_ch[["trade_date", "expiry", "market_open_pt", "first_candle_open", "first_candle_ac_ex_diff_pt", "ce_first_candle_chg_pc", "ce_first_candle_high_pc", "ce_ltp", "ce_first_candle_open", "ce_first_candle_high", "ce_first_candle_low", "ce_first_candle_close", "ce_oi", "ce_oi_chg"]]
# print(call_max_ch.loc[call_max_ch.ce_first_candle_chg_pc >= .02])
success_pc = (call_max_ch.loc[call_max_ch.ce_first_candle_chg_pc >= .05]).shape[0] * 100 / call_max_ch.shape[0]
call_max_ch.loc[:,"ce_first_candle_chg_pc"] = call_max_ch["ce_first_candle_chg_pc"].apply('{:.2%}'.format)
call_max_ch.loc[:,"ce_first_candle_high_pc"] = call_max_ch["ce_first_candle_high_pc"].apply('{:.2%}'.format)
print(success_pc)
call_max_ch[["trade_date", "expiry", "market_open_pt", "first_candle_open", "first_candle_ac_ex_diff_pt", "ce_ac_ex_diff", "ce_first_candle_chg_pc", "ce_first_candle_high_pc", "ce_ltp"]]

before filtering size: 3444
market up size: 1624
min_premium_df size: 734
diff positive size: 575
min_volume size: 315
size: 171
74.85380116959064


,trade_date,expiry,market_open_pt,first_candle_open,first_candle_ac_ex_diff_pt,ce_ac_ex_diff,ce_first_candle_chg_pc,ce_first_candle_high_pc,ce_ltp
17850,2023-01-05 00:00:00,2023-01-05 00:00:00,65.1,18101.95,18.75577,-0.098798,-5.95%,1.27%,206.2
17800,2023-01-05 00:00:00,2023-01-05 00:00:00,65.1,18101.95,19.58488,-0.097721,-5.14%,0.21%,252.3
17900,2023-01-05 00:00:00,2023-01-05 00:00:00,65.1,18101.95,5.61658,-0.002726,-12.41%,0.00%,160.75
17550,2023-01-31 00:00:00,2023-02-02 00:00:00,64.2,17731.45,42.685755,-0.059093,10.02%,12.81%,235.45
17400,2023-01-31 00:00:00,2023-02-02 00:00:00,64.2,17731.45,18.051865,0.017129,-1.33%,1.07%,342.55
17500,2023-01-31 00:00:00,2023-02-02 00:00:00,64.2,17731.45,14.883395,0.044617,-1.87%,1.02%,269.9
17450,2023-02-01 00:00:00,2023-02-02 00:00:00,129.05,17811.6,87.2895,-0.242627,21.74%,30.03%,326.1
17600,2023-02-01 00:00:00,2023-02-02 00:00:00,129.05,17811.6,40.18713,-0.114331,10.20%,18.48%,225.0
17400,2023-02-01 00:00:00,2023-02-02 00:00:00,129.05,17811.6,20.12505,-0.046478,0.14%,2.58%,365.0
17650,2023-02-03 00:00:00,2023-02-09 00:00:00,110.0,17721.75,27.40497,-0.181773,16.59%,27.71%,145.0


In [10]:
call_max_ch[["trade_date", "expiry", "market_open_pt", "first_candle_open", "first_candle_ac_ex_diff_pt", "ce_ac_ex_diff", "ce_first_candle_chg_pc", "ce_first_candle_high_pc", "ce_ltp"]].to_csv("actual_results.csv")

In [40]:
first_green[["trade_date", "expiry", "ce_first_candle_chg_pc"]]

,trade_date,expiry,ce_first_candle_chg_pc
strike_price,,,
18250,2023-01-02,2023-01-05,0.022296
18650,2023-01-04,2023-01-05,0.076923
18700,2023-01-04,2023-01-05,0.041667
18750,2023-01-04,2023-01-05,0.130435
18800,2023-01-04,2023-01-05,0.190476
...,...,...,...
21850,2023-12-27,2023-12-28,0.059524
21950,2023-12-27,2023-12-28,0.189189
22000,2023-12-27,2023-12-28,0.028571


In [28]:
first_green[["trade_date", "expiry"]]

,trade_date,expiry
strike_price,,
18250,2023-01-02,2023-01-05
18650,2023-01-04,2023-01-05
18700,2023-01-04,2023-01-05
18750,2023-01-04,2023-01-05
18800,2023-01-04,2023-01-05
...,...,...
21850,2023-12-27,2023-12-28
21950,2023-12-27,2023-12-28
22000,2023-12-27,2023-12-28
